In [3]:
import os

In [4]:
from beir import util, LoggingHandler
from beir.datasets.data_loader import GenericDataLoader

/home/addullah/miniconda3/envs/myenv/lib/python3.12/site-packages/beir/util.py:2: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [5]:
os.environ["CUDA_VISIBLE_DEVICES"]= "6"
tokenizer_max_len = 512

In [6]:
dataset = "msmarco_tiny"

dataset_path = "../beir/datasets/msmarco_tiny/"
corpus_file = "tiny_collection.json"
queries_file = "topics.dl20.txt"
qrels_test_file = "qrels.dl20-passage.txt"
training_set = "msmarco_triples.train.tiny.tsv"


In [11]:
import collections
import pytrec_eval
import json

def load_queries(path):
    """Returns a dictionary whose keys are query ids and values are query texts."""
    queries = {}
    with open(path) as f:
        for line in f:
            query_id, query_text = line.strip().split('\t')
            queries[query_id] = query_text
    return queries


def load_corpus_tsv(path):
    """Returns a dictionary whose keys are passage ids and values are passage texts."""
    corpus = {}
    with open(path) as f:
        for line in f:
            passage_id, passage_text = line.strip().split('\t')
            corpus[passage_id] = {'text': passage_text}
    return corpus

def load_corpus_json(path):
    with open(path, 'r') as corpus_f:
        corpus_json = json.load(corpus_f)
    return corpus_json


def load_qrels(path):
    with open(path, 'r') as f_qrel:
        qrels = pytrec_eval.parse_qrel(f_qrel)

    return qrels


def load_triplets(path):
    triplets = []
    with open(path) as f:
        for line in f:
            query, positive_passage, negative_passage = line.strip().split('\t')
            triplets.append([query, positive_passage, negative_passage])
    return triplets

# Don't need to load triplet for training bm25
# triplets = load_triplets('msmarco_triples.train.tiny.tsv')

qrels = load_qrels(f"{dataset_path}{qrels_test_file}")
queries = load_queries(f"{dataset_path}{queries_file}")
print("Loading corpus into memory ...")
corpus = load_corpus_json(f"{dataset_path}{corpus_file}")

Loading corpus into memory ...


In [12]:
len(corpus)

510585

In [13]:
import torch
from transformers import AutoModel, AutoTokenizer
from peft import PeftModel, PeftConfig

/home/addullah/miniconda3/envs/myenv/lib/python3.12/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


def get_model(peft_model_name):
    config = PeftConfig.from_pretrained(peft_model_name)
    base_model = AutoModel.from_pretrained(config.base_model_name_or_path)
    model = PeftModel.from_pretrained(base_model, peft_model_name)
    model = model.merge_and_unload()
    model.eval()
    return model

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('meta-llama/Llama-2-7b-hf')
model = get_model('castorini/repllama-v1-7b-lora-passage')

model = model.to(device) # Moving model to GPU

Loading checkpoint shards: 100%|██████████| 2/2 [00:05<00:00,  2.70s/it]


In [10]:
# from datasets import Dataset
# import pandas as pd

# queries = pd.DataFrame({'qid': queries.keys(), 'text': queries.values()})
# # queries.head()
# query_dataset = Dataset.from_pandas(queries)
# query_dataset

Dataset({
    features: ['qid', 'text'],
    num_rows: 300
})

In [12]:
# if tokenizer.pad_token is None:
#     tokenizer.add_special_tokens({'pad_token': '[PAD]'})
#     model.resize_token_embeddings(len(tokenizer)) # Update models token embedding size so it knows about new token

In [25]:
# input_txt = [query_dataset[0]['text'], query_dataset[1]['text']]

# tokenized_inputs = tokenizer(input_txt, return_tensors='pt', padding=True, truncation=True, max_length=tokenizer_max_len)
# tokenized_inputs.to(device)
# with torch.no_grad():
#     # compute query embedding
#     outputs = model(**tokenized_inputs)
#     embedding = outputs.last_hidden_state[0][-1]
#     embedding = torch.nn.functional.normalize(embedding, p=2, dim=0)

# embedding

tensor([ 0.0002,  0.0066,  0.0047,  ..., -0.0012, -0.0056, -0.0186],
       device='cuda:0')

In [9]:
# Can't batch tokenize because we need embeddings of last token and last token of different models maybe different

def get_embed_dataset(input_lst):

    input_txt = [f'{input}</s>' for input in input_lst['text']]

    tokenized_inputs = tokenizer(input_txt, return_tensors='pt', padding="max_length", truncation=True, max_length=tokenizer_max_len)
    tokenized_inputs = tokenized_inputs.to(device)
    with torch.no_grad():
        # compute query embedding
        outputs = model(**tokenized_inputs)
        embedding = outputs.last_hidden_state[:,-1,:]   #outputs.last_hidden_state[0][-1] # Get embedding of last token i.e. <s>
        embedding = torch.nn.functional.normalize(embedding, p=2, dim=0)
    return embedding

def get_embed(input):

    tokenized_inputs = tokenizer(f'{input}</s>', return_tensors='pt')
    tokenized_inputs = tokenized_inputs.to(device)
    with torch.no_grad():
        # compute query embedding
        outputs = model(**tokenized_inputs)
        embedding = outputs.last_hidden_state[0][-1] #outputs.last_hidden_state[:,-1,:]  # Get embedding of last token i.e. </s>
        embedding = torch.nn.functional.normalize(embedding, p=2, dim=0)
    return embedding

# query_dataset = query_dataset.map(get_embed, batched=True, batch_size=8)
# query_dataset

In [10]:
# from tqdm import tqdm

# query_embeddings = {}
# doc_embeddings = {}

# print("Encoding queries ...")
# for k,q in tqdm(queries.items()):
#     query_embed = get_embed(q)
#     query_embeddings[k] = query_embed

# print("Encoding passages ...")
# for k,q in tqdm(corpus.items()):
#     doc_embed = get_embed(q['text'])
#     doc_embeddings[k] = doc_embed

# with open(f"{dataset_path}queries_llamaEmbed.json", 'w') as f:
#     json.dump(query_embeddings, f)

# with open(f"{dataset_path}tiny_collection_llamaEmbed.json", 'w') as f:
#     json.dump(doc_embeddings, f)

Encoding queries ...


100%|██████████| 54/54 [00:04<00:00, 11.37it/s]


Encoding passages ...


  0%|          | 2184/510585 [03:44<14:31:54,  9.72it/s]


KeyboardInterrupt: 

In [8]:
import pickle

with open(f"{dataset_path}tiny_collection_llamaEmbed.pickle", 'rb') as f:
    doc_embeddings = pickle.load(f)

In [9]:
with open(f"{dataset_path}queries_llamaEmbed.pickle", 'rb') as f:
    query_embeddings = pickle.load(f)

In [14]:
from tqdm import tqdm

results = {}
for q_id, q_embed in tqdm(query_embeddings.items()):
    results[q_id] = {}
    for d_id, d_embed in doc_embeddings.items():
        # compute similarity score
        score = torch.dot(q_embed, d_embed)
        results[q_id][d_id] = score.item() #.item() to get value out of tensor


100%|██████████| 54/54 [11:47<00:00, 13.11s/it]


In [60]:
import pytrec_eval

metric = 'ndcg_cut_10'
evaluator = pytrec_eval.RelevanceEvaluator(qrels, {metric})
results_metric = evaluator.evaluate(results)
print(f'nDCG@10: {sum(item[metric] for item in results_metric.values()) / len(results_metric):0.4f}')

nDCG@10: 0.7599


In [61]:
metric = 'map_cut_10'
evaluator = pytrec_eval.RelevanceEvaluator(qrels, {metric})
results_metric = evaluator.evaluate(results)
print(f'MAP@10: {sum(item[metric] for item in results_metric.values()) / len(results_metric):0.4f}')

MAP@10: 0.7110


In [15]:
from beir.retrieval.search.lexical import BM25Search as BM25
from beir.retrieval.evaluation import EvaluateRetrieval

hostname = "localhost"
index_name = "scifact"
initialize = True

model = BM25(index_name=index_name, hostname=hostname, initialize=initialize)
retriever = EvaluateRetrieval(model)
# model doesn't do anything and results for ndcg and map are same as pytrec_eval

retriever.evaluate(qrels, results, retriever.k_values)

ERROR:root:Unable to create Index in Elastic Search. Reason: ConnectionError(<urllib3.connection.HTTPConnection object at 0x7f19a3f09970>: Failed to establish a new connection: [Errno 111] Connection refused) caused by: NewConnectionError(<urllib3.connection.HTTPConnection object at 0x7f19a3f09970>: Failed to establish a new connection: [Errno 111] Connection refused)
ERROR:root:Unable to create Index in Elastic Search. Reason: ConnectionError(<urllib3.connection.HTTPConnection object at 0x7f19a3f0a180>: Failed to establish a new connection: [Errno 111] Connection refused) caused by: NewConnectionError(<urllib3.connection.HTTPConnection object at 0x7f19a3f0a180>: Failed to establish a new connection: [Errno 111] Connection refused)


({'NDCG@1': 0.78395,
  'NDCG@3': 0.74971,
  'NDCG@5': 0.72805,
  'NDCG@10': 0.70889,
  'NDCG@100': 0.71979,
  'NDCG@1000': 0.80971},
 {'MAP@1': 0.03959,
  'MAP@3': 0.08449,
  'MAP@5': 0.12773,
  'MAP@10': 0.21312,
  'MAP@100': 0.53599,
  'MAP@1000': 0.60928},
 {'Recall@1': 0.03959,
  'Recall@3': 0.08614,
  'Recall@5': 0.13913,
  'Recall@10': 0.24466,
  'Recall@100': 0.71407,
  'Recall@1000': 0.9321},
 {'P@1': 0.88889,
  'P@3': 0.83951,
  'P@5': 0.8,
  'P@10': 0.76296,
  'P@100': 0.38333,
  'P@1000': 0.06098})

In [68]:
metric = 'recall.10'
evaluator = pytrec_eval.RelevanceEvaluator(qrels, {metric})
results_metric = evaluator.evaluate(results)
# print(f'recall@10: {sum(item[metric] for item in results_metric.values()) / len(results_metric):0.4f}')
results_metric

{'1': {'recall_10': 0.0},
 '3': {'recall_10': 1.0},
 '5': {'recall_10': 1.0},
 '13': {'recall_10': 0.0},
 '36': {'recall_10': 0.5},
 '42': {'recall_10': 1.0},
 '48': {'recall_10': 1.0},
 '49': {'recall_10': 1.0},
 '50': {'recall_10': 1.0},
 '51': {'recall_10': 1.0},
 '53': {'recall_10': 1.0},
 '54': {'recall_10': 1.0},
 '56': {'recall_10': 1.0},
 '57': {'recall_10': 1.0},
 '70': {'recall_10': 1.0},
 '72': {'recall_10': 1.0},
 '75': {'recall_10': 1.0},
 '94': {'recall_10': 1.0},
 '99': {'recall_10': 1.0},
 '100': {'recall_10': 1.0},
 '113': {'recall_10': 1.0},
 '115': {'recall_10': 1.0},
 '118': {'recall_10': 1.0},
 '124': {'recall_10': 1.0},
 '127': {'recall_10': 1.0},
 '128': {'recall_10': 0.0},
 '129': {'recall_10': 1.0},
 '130': {'recall_10': 1.0},
 '132': {'recall_10': 0.0},
 '133': {'recall_10': 0.8},
 '137': {'recall_10': 1.0},
 '141': {'recall_10': 1.0},
 '142': {'recall_10': 1.0},
 '143': {'recall_10': 1.0},
 '146': {'recall_10': 1.0},
 '148': {'recall_10': 1.0},
 '163': {'reca

In [69]:
metric = 'P.10'
evaluator = pytrec_eval.RelevanceEvaluator(qrels, {metric})
results_metric = evaluator.evaluate(results)
# print(f'P@10: {sum(item[metric] for item in results_metric.values()) / len(results_metric):0.4f}')
results_metric

{'1': {'P_10': 0.0},
 '3': {'P_10': 0.1},
 '5': {'P_10': 0.1},
 '13': {'P_10': 0.0},
 '36': {'P_10': 0.1},
 '42': {'P_10': 0.1},
 '48': {'P_10': 0.1},
 '49': {'P_10': 0.1},
 '50': {'P_10': 0.1},
 '51': {'P_10': 0.1},
 '53': {'P_10': 0.1},
 '54': {'P_10': 0.1},
 '56': {'P_10': 0.1},
 '57': {'P_10': 0.1},
 '70': {'P_10': 0.2},
 '72': {'P_10': 0.1},
 '75': {'P_10': 0.1},
 '94': {'P_10': 0.1},
 '99': {'P_10': 0.1},
 '100': {'P_10': 0.1},
 '113': {'P_10': 0.1},
 '115': {'P_10': 0.1},
 '118': {'P_10': 0.1},
 '124': {'P_10': 0.1},
 '127': {'P_10': 0.1},
 '128': {'P_10': 0.0},
 '129': {'P_10': 0.1},
 '130': {'P_10': 0.1},
 '132': {'P_10': 0.0},
 '133': {'P_10': 0.4},
 '137': {'P_10': 0.1},
 '141': {'P_10': 0.2},
 '142': {'P_10': 0.1},
 '143': {'P_10': 0.1},
 '146': {'P_10': 0.1},
 '148': {'P_10': 0.1},
 '163': {'P_10': 0.1},
 '171': {'P_10': 0.1},
 '179': {'P_10': 0.4},
 '180': {'P_10': 0.1},
 '183': {'P_10': 0.1},
 '185': {'P_10': 0.1},
 '198': {'P_10': 0.1},
 '208': {'P_10': 0.1},
 '212': {'